In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 762.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


## **Testing if able to download datasets**

In [4]:
#Import wildlife-datasets Library
from wildlife_datasets import datasets, splits

#Download dataset
datasets.ATRW.get_data('data/ATRW')

#Load metadata
metadata = datasets.ATRW('data/ATRW')

#Get 80/20 training/test split
splitter = splits.ClosedSetSplit(0.8)
splitter.split(metadata.df)


DATASET ATRW: DOWNLOADING STARTED.


atrw_detection_test.tar.gz: 1.49GB [01:03, 23.6MB/s]                            
atrw_reid_train.tar.gz: 139MB [00:05, 24.4MB/s]                           
atrw_anno_reid_train.tar.gz: 106kB [00:00, 327kB/s]                              
atrw_reid_test.tar.gz: 96.9MB [00:03, 24.8MB/s]                            
atrw_anno_reid_test.tar.gz: 90.1kB [00:00, 343kB/s]                             
main.zip: 30.7MB [00:09, 3.36MB/s]


DATASET ATRW: EXTRACTING STARTED.
DATASET ATRW: FINISHED.



[(array([4568, 3025, 1197, ..., 2301, 4525, 2355]),
  array([2199,  612,  634, ..., 4703,  142, 1124]))]

# **SIFT**

# **Acrface_Loss accuracy**

In [5]:
import itertools
from itertools import chain

import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits

from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

2024-08-02 20:37:25.116382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 20:37:25.116491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 20:37:25.241292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Training IPanda50

In [6]:

# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=30,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()


DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=616aeeb8-fdc9-481f-b15b-0671fc10f17d
To: /kaggle/data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:06<00:00, 146MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /kaggle/data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 68.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /kaggle/data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 177MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

...

Epoch 28: Average Loss = 1.576590657234192


Epoch 28: 100%|█████████████████████████████████████████████████████| 54/54 [01:23<00:00,  1.55s/it]


Epoch 29: Average Loss = 1.8520921468734741


Epoch 29: 100%|█████████████████████████████████████████████████████| 54/54 [01:23<00:00,  1.56s/it]

Epoch 30: Average Loss = 1.6566685438156128


In [7]:
import torch

In [8]:
trainer.save("retrained_chks", file_name="arcfaceloass_IPanda50_retrained_checkpoint.pth")

In [9]:
# # Assuming you need to use the model for further operations
# # Initialize the model architecture again as needed for loading
# # Make sure to initialize it exactly as you did for training
# model = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=False)

# # # Create a BasicTrainer instance with the initialized model (other parameters should match the training setup)
# # trainer = BasicTrainer(
# #     dataset=None,  # Assuming no dataset is needed just for loading
# #     model=model,
# #     objective=None,  # Assuming no objective needed just for loading
# #     optimizer=None,  # Assuming no optimizer needed just for loading
# #     epochs=0,  # No training epochs needed for just loading
# #     device='cuda'  # Adjust as per your device configuration
# # )

# trainer = BasicTrainer(
#     dataset=dataset,
#     model=backbone,
#     objective=objective,
#     optimizer=optimizer,
#     epochs=2,
#     device='cuda',
#     epoch_callback=print_epoch_loss
# )
# # Load the model
# trainer.load("retrained_chks/arcfaceloass_retrained_checkpoint.pth")

# # After loading, the model in the trainer instance is now the re-trained model
# # and is ready for use in further processing or inference


In [10]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [11]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 53/53 [14:40<00:00, 16.61s/it]


In [12]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.01373724, -0.01693645, -0.00358774, ...,  0.9742421 ,
        -0.02400762, -0.03572111],
       [ 0.10954875, -0.04768144, -0.00451115, ..., -0.05155712,
        -0.10766761, -0.01568238],
       [ 0.02020039, -0.01074846,  0.09873386, ..., -0.03288786,
         0.00351164, -0.09548655],
       ...,
       [ 0.00697668, -0.01339691,  0.05396191, ..., -0.02497772,
        -0.00802807, -0.05362709],
       [-0.01655747,  0.024214  , -0.02615532, ..., -0.12847045,
         0.01572288, -0.00589841],
       [-0.03794417, -0.07560731,  0.9013165 , ..., -0.01055363,
        -0.0507232 , -0.08384195]], dtype=float32)}


In [13]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['36_xingda' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '36_xingda' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '36_xingda' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '28_sa' '29_shuangxiong'
 '46_yazhu' '44_yayi' '14_maodou' '30_shuqing' '36_xingda' '39_xinghui'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '37_xinger'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongba

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


TRIPLET_LOSS

In [14]:
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer , TripletLoss

In [15]:


# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()



DATASET IPanda50: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


Epoch 0:   0%|                                                               | 0/54 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.39s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.57s/it]


Epoch 1: Average Loss = 0.245121568441391


Epoch 1: 100%|██████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.56s/it]


Epoch 2: Average Loss = 0.2331721931695938


...

Epoch 18: Average Loss = 0.19589205086231232


Epoch 18: 100%|█████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.56s/it]


Epoch 19: Average Loss = 0.1970953643321991


Epoch 19: 100%|█████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.56s/it]

Epoch 20: Average Loss = 0.19964082539081573


In [16]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_IPanda50_retrained_checkpoint.pth")




In [17]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


100%|███████████████████████████████████████████████████████████████| 53/53 [01:00<00:00,  1.14s/it]


In [18]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)


{'cosine': array([[ 0.00725734,  0.4591341 ,  0.07825134, ...,  0.41574007,
         0.40261596,  0.01987165],
       [ 0.04553113,  0.47067708, -0.0557644 , ...,  0.14869562,
         0.59519506, -0.14384368],
       [-0.04424645,  0.04630635,  0.11701504, ...,  0.02942031,
         0.20453586, -0.1253828 ],
       ...,
       [ 0.1135495 ,  0.1408934 , -0.04747616, ...,  0.015981  ,
         0.12062274, -0.09617898],
       [-0.09747559,  0.02062405,  0.04569775, ..., -0.1463177 ,
         0.10499872,  0.03330991],
       [-0.01803307,  0.11700951,  0.7057756 , ...,  0.27263647,
         0.04755937,  0.03263355]], dtype=float32)}


In [19]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '44_yayi' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '36_xingda' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '34_xiaoqiao' '35_xilan' '08_fulai' '00_aibang' '28_sa' '29_shuangxiong'
 '46_yazhu' '36_xingda' '14_maodou' '30_shuqing' '36_xingda' '39_xinghui'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '10_hexing'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongba

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


**NEW DATASET  DogFaceNet**

Arcface

In [20]:
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [21]:
import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer



# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()


DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:01, 43.2MB/s]                            


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



Epoch 0:   0%|                                                               | 0/66 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|█████████████████████████████████████████████████████▏| 65/66 [01:40<00:01,  1.54s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 66/66 [01:41<00:00,  1.53s/it]


Epoch 1: Average Loss = 39.466033935546875


...

Epoch 18: Average Loss = 27.780954360961914


Epoch 18: 100%|█████████████████████████████████████████████████████| 66/66 [01:41<00:00,  1.53s/it]


Epoch 19: Average Loss = 26.996768951416016


Epoch 19: 100%|█████████████████████████████████████████████████████| 66/66 [01:40<00:00,  1.53s/it]

Epoch 20: Average Loss = 26.420412063598633


In [22]:

trainer.save("retrained_chks", file_name="arcfaceloass_DogFaceNet_retrained_checkpoint.pth")

In [23]:
import torch

In [24]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [25]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 65/65 [16:38<00:00, 15.36s/it]


In [26]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-0.10120017,  0.00094893,  0.03113229, ...,  0.04185998,
        -0.01420362, -0.05570815],
       [ 0.01844995, -0.05779577, -0.01275181, ..., -0.11070522,
         0.03804523, -0.06749505],
       [ 0.0304631 ,  0.00978479,  0.02286835, ...,  0.00567573,
         0.03178079,  0.07905479],
       ...,
       [ 0.11919957, -0.08299096, -0.02329023, ...,  0.04016837,
        -0.0122668 ,  0.02739859],
       [ 0.07992694,  0.11752124,  0.05448681, ...,  0.0214443 ,
         0.02279913,  0.08330725],
       [ 0.14088176, -0.08075353, -0.03615448, ..., -0.04716393,
         0.08967818, -0.01942327]], dtype=float32)}


In [27]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['1249' '182' '919' '1122' '428' '378' '1063' '426' '470' '405' '764'
 '1090' '3' '1339' '432' '673' '1075' '274' '296' '234' '1039' '991' '485'
 '425' '280' '407' '324' '1248' '446' '896' '892' '448' '240' '251' '448'
 '930' '223' '210' '345' '1266' '622' '1324' '158' '127' '313' '1254'
 '1139' '208' '901' '424' '293' '415' '99' '147' '373' '192' '333' '333'
 '1314' '957' '329' '89' '189' '100' '122' '631' '367' '646' '26' '1369'
 '878' '673' '174' '1276' '1169' '458' '473' '149' '373' '622' '540' '919'
 '589' '913' '782' '1357' '361' '173' '1324' '953' '183' '200' '299' '191'
 '397' '187' '270' '563' '1020' '1074']
Accuracy on IPanda50 data: 90.00%
Precision: 1.0
Recall: 0.9
F1 Score: 0.9033333333333334


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T
